In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Model
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('blackjack_dataset.csv')

# Define a helper function to encode cards
def encode_card(card):
    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
    suit = card.split(' of ')[1]
    rank = card.split(' of ')[0]
    suit_idx = suits.index(suit)
    rank_idx = ranks.index(rank)
    return suit_idx * 13 + rank_idx

# Encode player hand and house upcard
df['Player Hand'] = df['Player Hand'].apply(eval)
df['Player Hand Encoded'] = df['Player Hand'].apply(lambda hand: [encode_card(card) for card in hand])
df['House Upcard Encoded'] = df['House Upcard'].apply(encode_card)

# Normalize and one-hot encode the cards
num_cards = 52
def one_hot_encode(cards, num_classes):
    encoding = np.zeros((num_classes,))
    for card in cards:
        encoding[card] = 1
    return encoding

df['Player Hand One-Hot'] = df['Player Hand Encoded'].apply(lambda hand: one_hot_encode(hand, num_cards))
df['House Upcard One-Hot'] = df['House Upcard Encoded'].apply(lambda card: one_hot_encode([card], num_cards))

# Combine features into a single input vector
df['Input Features'] = df.apply(lambda row: np.concatenate([row['Player Hand One-Hot'], row['House Upcard One-Hot'], [row['Count']]]), axis=1)

# Check the number of unique move categories
unique_moves = df['Move'].nunique()
print(f"Number of unique moves: {unique_moves}")

# Define the input shape
input_shape = df['Input Features'].iloc[0].shape

# Define the model
inputs = layers.Input(shape=input_shape)
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)

# Output for move prediction
move_output = layers.Dense(unique_moves, activation='softmax', name='move_output')(x)

# Output for bet prediction
bet_output = layers.Dense(1, activation='linear', name='bet_output')(x)

# Combine the model
model = Model(inputs=inputs, outputs=[move_output, bet_output])

model.compile(optimizer='adam',
              loss={'move_output': 'categorical_crossentropy', 'bet_output': 'mse'},
              metrics={'move_output': 'accuracy', 'bet_output': 'mae'})

# Convert input features and targets to numpy arrays
X = np.stack(df['Input Features'].values)
y_move = pd.get_dummies(df['Move']).values  # Assuming 'Move' is categorical
y_bet = df['Bet'].values

# Split the data
X_train, X_test, y_move_train, y_move_test, y_bet_train, y_bet_test = train_test_split(X, y_move, y_bet, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, {'move_output': y_move_train, 'bet_output': y_bet_train}, epochs=50, validation_split=0.2)

# Evaluate the model
results = model.evaluate(X_test, {'move_output': y_move_test, 'bet_output': y_bet_test})
print(f"Test results - Move Loss: {results[1]}, Move Accuracy: {results[3]}, Bet Loss: {results[2]}, Bet MAE: {results[4]}")


Number of unique moves: 5
Epoch 1/50
200/200 [==============================] - 3s 7ms/step - loss: 4261.2334 - move_output_loss: 1.5073 - bet_output_loss: 4259.7261 - move_output_accuracy: 0.4161 - bet_output_mae: 35.5752 - val_loss: 3256.0093 - val_move_output_loss: 1.3443 - val_bet_output_loss: 3254.6650 - val_move_output_accuracy: 0.4112 - val_bet_output_mae: 33.3221
Epoch 2/50
200/200 [==============================] - 1s 3ms/step - loss: 3265.9463 - move_output_loss: 1.3450 - bet_output_loss: 3264.6028 - move_output_accuracy: 0.4255 - bet_output_mae: 31.6468 - val_loss: 2995.6582 - val_move_output_loss: 1.3269 - val_bet_output_loss: 2994.3313 - val_move_output_accuracy: 0.4406 - val_bet_output_mae: 27.9922
Epoch 3/50
200/200 [==============================] - 1s 3ms/step - loss: 3027.3770 - move_output_loss: 1.2828 - bet_output_loss: 3026.0945 - move_output_accuracy: 0.4387 - bet_output_mae: 28.0059 - val_loss: 2863.3669 - val_move_output_loss: 1.3055 - val_bet_output_loss: 2862.